In [45]:
import pandas as pd

In [46]:
df = pd.read_csv("data.csv")

- first country
- duration in min
- remove empty budgets, revenue
- budget starts with $
- genres: drama, comedy, horror, docu, thriller, action, romance, western

In [47]:
df_small = df.dropna(
    subset=["country", "budget", "revenue", "duration"]
)
df_small = df_small[df_small["genres"].isin(["Comedy", "Drama", "Horror", "Documentary", "Thriller", "Action", "Romance", "Western"])]

In [48]:
df_small

,imdb_id,title,genres,release_date,country,production_companies,budget,revenue,duration,actors,director
4,113041,Father of the Bride Part II (1995),Comedy,"Release dateDecember 8, 1995 (United States)",Country of originUnited States,Touchstone Pictures;Sandollar Productions;Tayl...,"$30,000,000 (estimated)","$11,134,978",1 hr 46 min,Steve Martin;Diane Keaton;Martin Short;Kimberl...,Charles Shyer
12,113987,Nixon (1995),Drama,"Release dateJanuary 5, 1996 (United States)",Country of originUnited States,Hollywood Pictures;Illusion Entertainment;Cine...,"$44,000,000 (estimated)","$2,206,506",3 hr 12 min,Anthony Hopkins;Joan Allen;Powers Boothe;Ed Ha...,Oliver Stone
16,112281,Ace Ventura: When Nature Calls (1995),Comedy,"Release dateNovember 10, 1995 (United States)",Country of originUnited States,Morgan Creek Entertainment;Morgan Creek Produc...,"$30,000,000 (estimated)","$37,804,076",1 hr 30 min,Jim Carrey;Ian McNeice;Simon Callow;Maynard Ez...,Steve Oedekerk
23,114057,Othello (1995),Drama,"Release dateJanuary 19, 1996 (United States)",Countries of originUnited StatesUnited Kingdom,Castle Rock Entertainment;Dakota Films;Imminen...,"$11,000,000 (estimated)","$29,097",2 hr 3 min,Laurence Fishburne;Kenneth Branagh;Irène Jacob...,Oliver Parker
28,112792,Dangerous Minds (1995),Drama,"Release dateAugust 11, 1995 (United States)",Country of originUnited States,Hollywood Pictures;Don Simpson/Jerry Bruckheim...,"$23,000,000 (estimated)","$14,931,503",1 hr 39 min,Michelle Pfeiffer;George Dzundza;Courtney B. V...,John N. Smith
...,...,...,...,...,...,...,...,...,...,...,...
20439,2340784,"Last of the Unjust, The (Dernier des injustes,...",Documentary,"Release dateNovember 13, 2013 (France)",Countries of originFranceAustria,Synecdoche;Le Pacte;Dor Film Produktionsgesell...,"€2,500,000 (estimated)","$14,328",3 hr 40 min,Benjamin Murmelstein;Claude Lanzmann,Claude Lanzmann
20464,1992193,Pandora's Promise (2013),Documentary,"Release dateNovember 15, 2013 (United Kingdom)",Country of originUnited States,Robert Stone Productions;Vulcan Productions,"$1,000,000 (estimated)","$23,419",1 hr 27 min,Stewart Brand;Richard Rhodes;Gwyneth Cravens;M...,Robert Stone
20532,2170299,Bad Words (2013),Comedy,"Release dateMarch 28, 2014 (United States)",Country of originUnited States,Aggregate Films;Darko Entertainment;Digital Im...,"$10,000,000 (estimated)","$113,301",1 hr 29 min,Jason Bateman;Kathryn Hahn;Allison Janney;Roha...,Jason Bateman
20533,3210686,Son of God (2014),Drama,"Release dateFebruary 28, 2014 (United States)",Country of originUnited States,Hearst Entertainment Productions;K Films;Light...,"$22,000,000 (estimated)","$25,601,865",2 hr 18 min,Diogo Morgado;Amber Rose Revah;Greg Hicks;Adri...,Christopher Spencer


In [49]:
df_small["month"] = df_small["release_date"].str.extract(r"(January|February|March|April|May|June|July|August|September|October|November|December)")

In [50]:
df_small["date"] = df_small["release_date"].str.extract(r"(\d\d?)")

In [51]:
df_small["year"] = df_small["release_date"].str.extract(r"(\d{4})")

In [52]:
df_small["budget"] = df_small["budget"].str.extract(r"\$([\d,]+)")
df_small = df_small.dropna(subset=["budget"])
df_small["budget"] = df_small["budget"].astype(str).apply(lambda b: int(b.replace(",", "")))

<ipython-input-52-04b0f181f420>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small["budget"] = df_small["budget"].astype(str).apply(lambda b: int(b.replace(",", "")))


In [53]:
df_small["revenue"] = df_small["revenue"].str.extract(r"([\d,]+)")
df_small = df_small.dropna(subset=["revenue"])
df_small["revenue"] = df_small["revenue"].astype(str).apply(lambda r: int(r.replace(",", "")))

<ipython-input-53-118374751bcd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small["revenue"] = df_small["revenue"].str.extract(r"([\d,]+)")


In [54]:
import re
def convert_duration_to_min(d):
    times = re.findall("\d+", d)
    hr, minut = 0, 0
    if len(times) == 2:
        hr, minut = times
    elif "hr" in d:
        hr = times[0]
    elif "min" in d:
        minut = times[0]
    return int(hr) * 60 + int(minut)

In [55]:
df_small["duration"] = df_small["duration"].apply(convert_duration_to_min)

In [56]:
def get_main_country(countrystring):
    dual = False
    for i in range(len(list(countrystring))-1):
        if list(countrystring)[i].islower() and list(countrystring)[i+1].isupper():
            dual = True
    if dual == True:
        return re.search("(.+?[a-z])(?=[A-Z])",countrystring).group(0)
    else:
        return countrystring

In [57]:
df_small["country"] = (
    df_small["country"].str.replace("Country of origin", "")
    .str.replace("Countries of origin", "")
    .apply(get_main_country)
    .apply(lambda c: 1 if c != "United States" else 0)
)

In [58]:
df_small = df_small.rename({"country": "is_foreign"}, axis=1)

In [59]:
df_small = df_small.drop("release_date", axis=1)

In [60]:
df_small.shape

(1420, 13)

In [61]:
df_small["genres"].value_counts()

Drama          629
Comedy         519
Horror          78
Documentary     69
Thriller        60
Action          35
Romance         17
Western         13
Name: genres, dtype: int64

In [62]:
df_small.reset_index(drop=True).to_csv("data_clean.csv")

In [63]:
df = pd.read_csv("data_clean.csv", index_col=0)

In [64]:
df

,imdb_id,title,genres,is_foreign,production_companies,budget,revenue,duration,actors,director,month,date,year
0,113041,Father of the Bride Part II (1995),Comedy,0,Touchstone Pictures;Sandollar Productions;Tayl...,30000000,11134978,106,Steve Martin;Diane Keaton;Martin Short;Kimberl...,Charles Shyer,December,8,1995
1,113987,Nixon (1995),Drama,0,Hollywood Pictures;Illusion Entertainment;Cine...,44000000,2206506,192,Anthony Hopkins;Joan Allen;Powers Boothe;Ed Ha...,Oliver Stone,January,5,1996
2,112281,Ace Ventura: When Nature Calls (1995),Comedy,0,Morgan Creek Entertainment;Morgan Creek Produc...,30000000,37804076,90,Jim Carrey;Ian McNeice;Simon Callow;Maynard Ez...,Steve Oedekerk,November,10,1995
3,114057,Othello (1995),Drama,0,Castle Rock Entertainment;Dakota Films;Imminen...,11000000,29097,123,Laurence Fishburne;Kenneth Branagh;Irène Jacob...,Oliver Parker,January,19,1996
4,112792,Dangerous Minds (1995),Drama,0,Hollywood Pictures;Don Simpson/Jerry Bruckheim...,23000000,14931503,99,Michelle Pfeiffer;George Dzundza;Courtney B. V...,John N. Smith,August,11,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,2382396,Joe (2013),Drama,0,Worldview Entertainment;Dreambridge Films;Musk...,4000000,105881,117,Nicolas Cage;Tye Sheridan;Gary Poulter;Ronnie ...,David Gordon Green,April,11,2014
1416,2363471,"Summit, The (2012)",Documentary,1,Image Now;Fantastic Films;Passion Pictures,1000000,20904,95,Christine Barnes;Hoselito Bite;Walter Bonatti;...,Nick Ryan,November,22,2013
1417,1992193,Pandora's Promise (2013),Documentary,0,Robert Stone Productions;Vulcan Productions,1000000,23419,87,Stewart Brand;Richard Rhodes;Gwyneth Cravens;M...,Robert Stone,November,15,2013
1418,2170299,Bad Words (2013),Comedy,0,Aggregate Films;Darko Entertainment;Digital Im...,10000000,113301,89,Jason Bateman;Kathryn Hahn;Allison Janney;Roha...,Jason Bateman,March,28,2014
